In [15]:
# Selenium 라이브러리 설치
!pip install selenium
from selenium import webdriver

In [16]:
# 패키지와 드라이버 설치
import sys
!sudo add-apt-repository ppa:saiarcot895/chromium-beta
!sudo apt remove chromium-browser
!sudo snap remove chromium
!sudo apt install chromium-browser

!pip3 install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin/

PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/saiarcot895/chromium-beta/ubuntu/ jammy main'
Description:
This PPA contains the latest Chromium Beta builds, with hardware video decoding enabled (hidden behind a flag), and support for Widevine (needed for viewing many DRM-protected videos) enabled.

== Hardware Video Decoding ==

To enable hardware video decoding, start Chromium with the --enable-features=VaapiVideoDecoder argument. To make this persistent, create a file at /etc/chromium-browser/customizations/92-vaapi-hardware-decoding with the following contents:

CHROMIUM_FLAGS="${CHROMIUM_FLAGS} --enable-features=VaapiVideoDecoder"

See also https://wiki.archlinux.org/title/Chromium#Hardware_video_acceleration for more information on VAAPI video decoding support.

=== Widevine Support ===

The packages in this PPA have support for Widevine inside Chromium enabled. However, you still need to copy some files from 

In [17]:
progress_file = "/content/drive/MyDrive/[2024-1] EURON 6기/프로젝트/data/progress.json"
store_dict = []

In [125]:
# Selenium 관련 환경 설정
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, InvalidSessionIdException
from time import sleep
from bs4 import BeautifulSoup
import re
import json

sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('disable-dev-shm-usage')
options.add_experimental_option("detach", True)

webdriver_service = Service('/usr/bin/chromedriver')
driver = webdriver.Chrome(service=webdriver_service, options=options)

In [126]:
# 웹페이지 파싱
url = "https://map.naver.com/v5/search"
driver.get(url)
key_word = '신촌 맛집'
print(driver)

<selenium.webdriver.chrome.webdriver.WebDriver (session="8ab672980a8254c832b1a6dd2b860ceb")>


In [21]:
# css 찾을때 까지 10초 대기
def time_wait(num, code):
  try:
    wait = WebDriverWait(driver, num).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, code)))
  except:
    print(code, '태그를 찾지 못하였습니다.')
    driver.quit()
  return wait

In [22]:
# frame 변경 메소드
def switch_frame(frame):
  driver.switch_to.default_content()  # frame 초기화
  sleep(1)
  driver.switch_to.frame(frame)  # frame 변경
  res
  soup

In [23]:
# 페이지 다운
def page_down(num):
  body = driver.find_element(By.CSS_SELECTOR, 'body')
  body.click()
  for i in range(num):
    body.send_keys(Keys.PAGE_DOWN)

In [24]:
def save_progress(data):
    with open(progress_file, 'w') as f:
        json.dump(data, f)

def load_progress():
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            return json.load(f)
    return {"last_page": 1, "last_store": 0}

In [129]:
time_wait(10, 'div.input_box > input.input_search')

# (1) 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

sleep(10)

res = driver.page_source  # 페이지 소스 가져오기
soup = BeautifulSoup(res, 'html.parser')  # html 파싱

# (2) frame 변경 & 페이지 스크롤 다운
switch_frame("searchIframe")
page_down(60)
sleep(5)

In [ ]:
# 시작시간
progress = load_progress()
last_page = progress['last_page']
last_store = progress['last_store']

next_btn = driver.find_elements(By.CSS_SELECTOR, '.mBN2s')

start = time.time()
print('[크롤링 시작...]')

try:
    for btn in range(last_page, len(next_btn)+1):
        store_list = WebDriverWait(driver, 10).until(
                  EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.UEzoS')))
        print(len(store_list))
        for data in range(last_store, len(store_list)):  # 매장 리스트 만큼 반복
            # (3) 상세 페이지로 이동 (페이지 클릭)
            try:
                page = driver.find_elements(By.CSS_SELECTOR, '.TYaxT')
                if data < len(page):
                    page[data].click()
                    time.sleep(2)

                switch_frame('entryIframe')
                time_wait(5, '.GHAhO')

                # 페이지 스크롤 다운
                for _ in range(2):
                    time.sleep(1)
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # ----- (4) 매장명 & 장소 유형 -----
                store_name = driver.find_element(By.CSS_SELECTOR, '.GHAhO').text
                store_type = driver.find_element(By.CSS_SELECTOR, '.lnJFt').text

                # ----- (5) 별점 -----
                try:
                    store_rating = driver.find_element(By.CSS_SELECTOR, '.PXMot.LXIwF').text
                except:
                    store_rating = -1

                # ----- (6) 리뷰 탭으로 이동 -----
                try:
                    # veBoZ 클래스를 가진 모든 요소 찾기
                    tabs = driver.find_elements(By.CSS_SELECTOR, '.veBoZ')

                    # 각 탭 요소를 확인하여 텍스트가 "리뷰"인 탭 클릭
                    for tab in tabs:
                        if tab.text == "리뷰":
                            tab.click()
                            time.sleep(2)
                            break

                    # ----- (7) 대표 키워드 1 -----
                    keyword1 = driver.find_elements(By.CSS_SELECTOR, '.t3JSf')[0].text

                    # ----- (7) 대표 키워드 2 -----
                    keyword2 = driver.find_elements(By.CSS_SELECTOR, '.t3JSf')[1].text

                    # ----- 방문자 리뷰 -----
                    all_reviews = []
                    try:
                        # (8) 더보기 버튼을 클릭하여 리뷰 추가로 로드 - 더보기 한 번 당 10개의 리뷰
                        for _ in range(5):
                            try:
                                driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
                                sleep(3)
                                driver.find_element(By.CSS_SELECTOR, '.fvwqf').click()
                                sleep(3)
                                driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)

                            except NoSuchElementException:
                                print('-더보기 버튼 모두 클릭 완료-')
                                break

                        reviews = WebDriverWait(driver, 10).until(
                                      EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.zPfVt')))

                        # 각 리뷰 텍스트를 가져와서 리스트에 추가
                        for review in reviews:
                            try:
                                # (9)-a. 토글 버튼 클릭하여 전체 리뷰 텍스트 표시
                                toggle_button = review.find_element(By.CSS_SELECTOR, '.Ky28p')
                                toggle_button.click()
                                time.sleep(2)  # 토글 버튼 클릭 후 잠시 대기
                                full_text = review.find_element(By.CSS_SELECTOR, '.zPfVt').text
                                all_reviews.append(full_text)
                            except:
                                # (9)-b. 토글 버튼이 없는 경우 그냥 리뷰 텍스트 가져오기
                                text = review.text
                                all_reviews.append(text)

                    except:
                        all_reviews.append(' ')

                except:
                    print("리뷰탭을 찾지 못함")

                # 매장 정보 딕셔너리에 추가
                dict_temp = {
                              'name': store_name,
                              'type': store_type,
                              'rating' : store_rating,
                              '1st_keyword' : keyword1,
                              '2ed_keyword' : keyword2,
                              'review' : all_reviews
                            }

                store_dict.append(dict_temp)
                print(f'{store_name} ...완료')
                save_progress({'last_page': last_page, 'last_store': data + 1})

                # 검색 iframe으로 전환
                switch_frame('searchIframe')
                time.sleep(1)

            except Exception as e:
                print(f"Error occurred while processing store data: {e}")
                save_progress({'last_page': last_page, 'last_store': data})
                break

        # (10) 다음 페이지 버튼 클릭
        try:
            next_button = driver.find_elements(By.CSS_SELECTOR, '.mBN2s')
            if next_button[btn+1]:
                next_button[btn+1].click()
                time.sleep(5)
                last_page += 1
                last_store = 0
                save_progress({'last_page': last_page, 'last_store': 0})
        except InvalidSessionIdException as e:
            print(f"Invalid session ID exception: {e}")
            break
        except Exception as e:
            print(f"Error occurred while clicking next button: {e}")
            break

finally:
    print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
    driver.quit()

In [81]:
import json

# json 파일로 저장
with open('/content/drive/MyDrive/[2024-1] EURON 6기/프로젝트/data/store_data.json', 'w', encoding='utf-8') as f:
    json.dump(store_dict, f, indent=4, ensure_ascii=False)

# JSON 파일 경로 설정
file_path = '/content/drive/MyDrive/[2024-1] EURON 6기/프로젝트/data/store_data.json'

# JSON 파일 열기
with open(file_path, 'r') as file:
    data = json.load(file)

In [82]:
import pandas as pd
df = pd.DataFrame(data)
df.to_csv("/content/drive/MyDrive/[2024-1] EURON 6기/프로젝트/data/store_data.csv", index=False)

In [83]:
df.head()

,name,type,rating,1st_keyword,2ed_keyword,review
0,탐복 현대백화점신촌점,한식,별점\n4.47,"""음식이 맛있어요""","""재료가 신선해요""",[저희 어머니가 삼삼한걸 좋아하셔서 음식점 고르는데에 1순위가 간이 세지않은 것이었...
1,더도이축산직영점 신촌점,"육류,고기요리",별점\n4.34,"""음식이 맛있어요""","""가성비가 좋아요""","[웨이팅이 길어서 얼마나 맛있나싶었는데, 기다릴만합니다..삼겹살이 존맛이네요..😹,..."
2,한강로칼국수 현대백화점 신촌점,"칼국수,만두",-1,"""음식이 맛있어요""","""친절해요""","[이전에 방문했을땐 보리밥 양이 엄청 많았었는데, 그때보다 양이 적어진 것 같아요...."
3,평안도식당 신촌점,"순대,순댓국",-1,"""음식이 맛있어요""","""양이 많아요""",[점심을 든든하게 먹고싶어 들른 평안도식당 매장이 넓고 쾌적한데 24시간 운영하는 ...
4,단정 현대백화점 신촌점,한식,-1,"""음식이 맛있어요""","""친절해요""","[요즘 핫한 솥밥,, 소고기 솥밥이랑 가지소보루 솥밥 시켜봤는데, 맛도 좋고 양도 ..."


In [84]:
df.tail(10)

,name,type,rating,1st_keyword,2ed_keyword,review
162,신촌부추곱창,"곱창,막창,양",-1,"""음식이 맛있어요""","""고기 질이 좋아요""","[신촌에 곱창집 중 이 집이 제일 맛있어요!! 특히 부추와 파김치, 백김치랑 같이 ..."
163,스타벅스 신촌점,카페,별점\n4.48,"""커피가 맛있어요""","""음료가 맛있어요""",[그동안 딸기 아사이레모네이드 많이 마셨는데 카페인 성분이 높다는걸 오늘 첨으로 알...
164,하남돼지집 신촌점,돼지고기구이,별점\n4.37,"""음식이 맛있어요""","""친절해요""",[ ]
165,신촌설렁탕,"곰탕,설렁탕",별점\n3.6,"""음식이 맛있어요""","""친절해요""",[김치 깍두기가 맛있었어요! 드시는 분들은 설렁탕 간을 평소보다 약하게 하시는 걸 ...
166,리브팜 신촌,"다이어트,샐러드",-1,"""재료가 신선해요""","""음식이 맛있어요""",[직접 기른 채소를 먹는 건강한 샐러드 가게예요\n네이버 예약하고 갔는데 굳이 예약...
167,풍국면 현대백화점 신촌점,국수,별점\n4.27,"""음식이 맛있어요""","""친절해요""",[ ]
168,고래고래신촌점,생선회,별점\n4.59,"""음식이 맛있어요""","""단체모임 하기 좋아요""",[예전보다 가격이 좀 올라서 아쉽습니다만 여전히 가성비 괜찮습니다. 회도 신선하고 ...
169,계백집 현대백화점 신촌점,한식,별점\n4.49,"""음식이 맛있어요""","""혼밥하기 좋아요""",[단골입니다 그런데 오늘 충전되냐 물으니 충전기없음 안된다셨는데 너무 성의없이 느껴...
170,Poke all day 포케&샐러드 신촌점,"다이어트,샐러드",-1,"""음식이 맛있어요""","""재료가 신선해요""","[평소 좋아하는 매장인데, 신촌에 생겨서 너무 좋아요🙂\n새로 생겨서 매장도 깨끗하..."
171,홍미닭발 신촌점,닭발,별점\n4.43,"""음식이 맛있어요""","""친절해요""",[이영자 입간판보고 기대를 많이 해서그런지 무뼈세트 양이 적어서 내심 놀랐어요 매운...


In [85]:
# JSON 파일 경로 설정
file_path = '/content/drive/MyDrive/[2024-1] EURON 6기/프로젝트/data/progress.json'

# JSON 파일 열기
with open(file_path, 'r') as file:
    progress = json.load(file)
progress

{'last_page': 5, 'last_store': 0}